# 1 perpare AKS

## load environment variables

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/container-centric-fiware-demonstration"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/container-centric-fiware-demonstration
```

In [ ]:
source ${PJ_ROOT}/docs/azure_aks/env

## login AKS

In [ ]:
az login --tenant ${TENANT}

## create DNS zone

In [ ]:
az group create --name ${DNS_ZONE_RG} --location ${REGION}

In [ ]:
az network dns zone create --resource-group ${DNS_ZONE_RG} --name "${DOMAIN}"

In [ ]:
az network dns zone show --resource-group ${DNS_ZONE_RG} --name "${DOMAIN}" | jq ".nameServers"

example)
```
[
  "ns1-XX.azure-dns.com.",
  "ns2-XX.azure-dns.net.",
  "ns3-XX.azure-dns.org.",
  "ns4-XX.azure-dns.info."
]
```

## create Resource Group for AKS

In [ ]:
az group create --name ${AKS_RG} --location ${REGION}

## start private registry on Azure Container Registry

In [ ]:
az acr create --resource-group ${AKS_RG} --name ${ACR_NAME} --sku Basic

In [ ]:
export REPOSITORY=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} | jq '.loginServer' -r)
sed -i '' -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/azure_aks/env
echo ${REPOSITORY}

example)
```
ccfdacr.azurecr.io
```

In [ ]:
az acr login --name ${ACR_NAME}

## start kubernetes on Azure AKS

### prepare AKS

In [ ]:
az provider register -n Microsoft.Compute

In [ ]:
az provider register -n Microsoft.Storage

In [ ]:
az provider register -n Microsoft.Network

In [ ]:
az provider register -n Microsoft.ContainerService

In [ ]:
az provider show -n Microsoft.Compute | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.Storage | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.Network | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.ContainerService | jq '.registrationState' -r

expected)
```
Registered
```

`az aks create` command takes a long time to execute (about 10~20 minutes).

**select `A. start AKS without monitoring & logging` or `B. start AKS with monitoring & logging`**

### A. start AKS without monitoring & logging

In [ ]:
az aks create --resource-group ${AKS_RG} --name ${AKS_NAME} --node-count ${NODE_COUNT} --ssh-key-value ${SSH_KEY} --kubernetes-version 1.11.2

### B. start AKS with monitoring & logging

In [ ]:
export NODE_VM_SIZE=Standard_D2s_v3
az aks create --resource-group ${AKS_RG} --name ${AKS_NAME} --node-count ${NODE_COUNT} --node-vm-size ${NODE_VM_SIZE} --ssh-key-value ${SSH_KEY} --kubernetes-version 1.11.2

### get credential of AKS

In [ ]:
az aks get-credentials --resource-group ${AKS_RG} --name ${AKS_NAME}

In [ ]:
CLIENT_ID=$(az aks show --resource-group ${AKS_RG} --name ${AKS_NAME} --query "servicePrincipalProfile.clientId" --output tsv)
ACR_ID=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} --query "id" --output tsv)
az role assignment create --assignee ${CLIENT_ID} --role Reader --scope ${ACR_ID}

### confirm nodes

In [ ]:
kubectl get nodes

example)
```
NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-22617196-0   Ready     agent     1d        v1.11.2
aks-nodepool1-22617196-1   Ready     agent     1d        v1.11.2
aks-nodepool1-22617196-2   Ready     agent     1d        v1.11.2
```

### setup rbac

In [ ]:
kubectl apply -f rbac/dashboard-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep kubernetes-dashboard

expected)
```
kubernetes-dashboard                 1         1m
```

In [ ]:
kubectl apply -f rbac/tiller-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep tiller

expected)
```
tiller                               1         1m
```

In [ ]:
kubectl apply -f rbac/default-rbac.yaml

In [ ]:
kubectl get clusterroles | grep default-read

expected)
```
default-read                                                           1m
```

In [ ]:
kubectl get clusterrolebindings | grep default

expected)
```
default                                                2m
```

## setup helm

In [ ]:
helm version --client

expected)
```
Client: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
```

In [ ]:
helm init --service-account tiller

In [ ]:
helm repo update

In [ ]:
kubectl get pods --all-namespaces | grep tiller

expected)
```
kube-system   tiller-deploy-7ccf99cd64-gkn7t          1/1       Running   0          1m
```

In [ ]:
helm version

expected)
```
Client: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.10.0", GitCommit:"9ad53aac42165a5fadc6c87be0dea6b115f93090", GitTreeState:"clean"}
```